<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Stuttgart</font></h1>

### Install and import libraries

In [4]:
#Some install

In [1]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.0       |   py36h9f0ad1d_0         160 KB  conda-forge
    soupsieve-1.9.4            |   py36h9f0ad1d_1          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         218 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     conda-forge/linux-64::beautifulsoup4-4.9.0-py36h9f0ad1d_0
  soupsieve          conda-forge/linux-64::soupsieve-1.9.4-py36h9f0ad1d_1



soupsieve-1.9.4      | 58 KB     | ##################################### | 100% 
beautifulsoup4-4.9.0 | 160 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction

In [2]:
!conda install -c conda-forge geopy --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install -c conda-forge geocoder --yes 

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    click-7.1.1                |     pyh8c360ce_0          64 KB  conda-forge
    cryptography-2.8           |   py36h45558ae_2         628 KB  conda-forge
    future-0.18.2              |   py36h9f0ad1d_1         714 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    urllib3-1.25.9             |             py_0          92 KB  conda-forge
    -

In [4]:
# Some libraries

from bs4 import BeautifulSoup
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import folium # plotting library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

### Data import and pre-processing

In [5]:
# Get all postal codes of Germany in csv format

raw_csv_link='https://raw.githubusercontent.com/zauberware/postal-codes-json-xml-csv/master/data/DE/zipcodes.de.csv'
raw_csv = pd.read_csv(raw_csv_link)
raw_csv.head()

,country_code,zipcode,place,state,state_code,province,province_code,community,community_code,latitude,longitude
0,DE,1945,Grünewald,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4000,14.0000
1,DE,1945,Lindenau,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4000,13.7333
2,DE,1945,Hohenbocka,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4310,14.0098
3,DE,1945,Schwarzbach,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4500,13.9333
4,DE,1945,Guteborn,Brandenburg,BB,NaN,0,Landkreis Oberspreewald-Lausitz,12066,51.4167,13.9333


In [6]:
# Only keep Stuttgart

raw_df=raw_csv[raw_csv['place'].str.contains('Stuttgart')]
raw_df.head()

,country_code,zipcode,place,state,state_code,province,province_code,community,community_code,latitude,longitude
1482,DE,70173,Stuttgart Stuttgart-Mitte,Baden-Württemberg,BW,Regierungsbezirk Stuttgart,81,Stuttgart,8111,48.7786,9.1767
1483,DE,70173,Stuttgart,Baden-Württemberg,BW,Regierungsbezirk Stuttgart,81,Stuttgart,8111,48.7667,9.1833
1484,DE,70174,Stuttgart Stuttgart-Nord,Baden-Württemberg,BW,Regierungsbezirk Stuttgart,81,Stuttgart,8111,48.7978,9.1921
1485,DE,70174,Stuttgart,Baden-Württemberg,BW,Regierungsbezirk Stuttgart,81,Stuttgart,8111,48.7667,9.1833
1486,DE,70174,Stuttgart Stuttgart-Mitte,Baden-Württemberg,BW,Regierungsbezirk Stuttgart,81,Stuttgart,8111,48.7827,9.1720


In [10]:
# Some data cleaning

df=raw_df[['zipcode','place','latitude','longitude']]
df=df.reset_index(drop=True)
df['place'].replace(regex='Stuttgart ', value='',inplace=True)
df=df.drop(df[df['place']=='Stuttgart'].index)
df = df.sort_values(by=['place'])
df=df.reset_index(drop=True)
df

,zipcode,place,latitude,longitude
0,70599,Asemwald,48.7260,9.1932
1,70376,Bad Cannstatt,48.8056,9.2141
2,70374,Bad Cannstatt,48.8056,9.2141
3,70372,Bad Cannstatt,48.8056,9.2141
4,70191,Bad Cannstatt,48.8056,9.2141
...,...,...,...,...
85,70435,Zuffenhausen,48.8334,9.1656
86,70437,Zuffenhausen,48.8353,9.1938
87,70469,Zuffenhausen,48.8141,9.1675
88,70499,Zuffenhausen,48.8323,9.1685


In [11]:
df['place'].nunique()

53

In [12]:
# Group by neighborhoods

df_unique=df
df['zipcode'] = df['zipcode'].astype(str)
for i in range(0,df.shape[0]-1):
    if df_unique.at[i,'place']==df_unique.at[i+1,'place']:
        df_unique.at[i+1,'zipcode']=df_unique.at[i+1,'zipcode']+', '+df_unique.at[i,'zipcode']
        df_unique=df_unique.drop([i])
df_unique.head()

,zipcode,place,latitude,longitude
0,70599,Asemwald,48.7260,9.1932
4,"70191, 70372, 70374, 70376",Bad Cannstatt,48.8056,9.2141
5,70499,Bergheim,48.8013,9.0897
6,70599,Birkach,48.7082,9.2108
7,70195,Botnang,48.7762,9.1304


In [13]:
df_unique.shape

(53, 4)

In [14]:
# Rename Columns and reset index

df_unique.rename(columns={'zipcode':'PostalCode','place':'Neighborhood','latitude':'Latitude','longitude':'Longitude'}, inplace=True)
df_Stuttgart=df_unique.reset_index(drop=True)
df_Stuttgart.head()

,PostalCode,Neighborhood,Latitude,Longitude
0,70599,Asemwald,48.7260,9.1932
1,"70191, 70372, 70374, 70376",Bad Cannstatt,48.8056,9.2141
2,70499,Bergheim,48.8013,9.0897
3,70599,Birkach,48.7082,9.2108
4,70195,Botnang,48.7762,9.1304


### Display Stuttgart and the neighborhoods

In [15]:
address = "Stuttgart"

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Stuttgart are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Stuttgart are 48.7784485, 9.1800132.


In [21]:
# create map of Stuttgart using latitude and longitude values
map_Stuttgart = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood, postalcode in zip(df_Stuttgart['Latitude'], df_Stuttgart['Longitude'], df_Stuttgart['Neighborhood'], df_Stuttgart['PostalCode']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Stuttgart)  
    
map_Stuttgart

In [17]:
CLIENT_ID = 'GS0MDHBH4F5O4AHPQLBMNJWQGPZ5DUCG4BAGCCTJQZUQAI2W' # your Foursquare ID
CLIENT_SECRET = 'HOWEWAL4YFF30AKLLRRRKSUIQMNTQSKGV3J1XXUKVWVAZSDS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GS0MDHBH4F5O4AHPQLBMNJWQGPZ5DUCG4BAGCCTJQZUQAI2W
CLIENT_SECRET:HOWEWAL4YFF30AKLLRRRKSUIQMNTQSKGV3J1XXUKVWVAZSDS


## Explore Neighborhoods in Stuttgart

Let's explore the food places in all neighborhoods

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius, categoryId):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            categoryId)
    
        # make the GET request
        results = requests.get(url).json()['response']['venues']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    print('Done')
    return(nearby_venues)

In [44]:
# Global parameters to use in the calls

nb=df_Stuttgart['Neighborhood']
lt=df_Stuttgart['Latitude']
lg=df_Stuttgart['Longitude']
rd=1000

In [45]:
# Make the resto call

df_Stuttgart_resto = getNearbyVenues(names=nb,
                                   latitudes=lt,
                                   longitudes=lg,
                                     radius=rd,
                                      categoryId='4d4b7105d754a06374d81259'
                                  )

Done


In [46]:
print(df_Stuttgart_resto.shape)
df_Stuttgart_resto.head()

(1198, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Asemwald,48.726,9.1932,Bella Vista Sky Restaurant Betriebe UG,48.725801,9.193008,Gastropub
1,Asemwald,48.726,9.1932,Broadway Cafe,48.723146,9.179163,Café
2,Asemwald,48.726,9.1932,Kantine Kelley,48.723230,9.177962,German Restaurant
3,Asemwald,48.726,9.1932,China Palast,48.727575,9.205185,Chinese Restaurant
4,Asemwald,48.726,9.1932,Kelly Food Court,48.722697,9.180638,American Restaurant


In [47]:
# Let's encode these data and group by neighborhood

# one hot encoding
df_Stuttgart_onehot_resto = pd.get_dummies(df_Stuttgart_resto[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_Stuttgart_onehot_resto['Neighborhood'] = df_Stuttgart_resto['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_Stuttgart_onehot_resto.columns[-1]] + list(df_Stuttgart_onehot_resto.columns[:-1])
df_Stuttgart_onehot_resto = df_Stuttgart_onehot_resto[fixed_columns]

df_Stuttgart_grouped_resto = df_Stuttgart_onehot_resto.groupby('Neighborhood').sum().reset_index()
df_Stuttgart_grouped_resto.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,BBQ Joint,Bakery,Bistro,...,Sushi Restaurant,Swabian Restaurant,Tapas Restaurant,Taverna,Tea Room,Thai Restaurant,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Wine Bar
0,Asemwald,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
1,Bad Cannstatt,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,1,0,0
2,Bergheim,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,Birkach,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0
4,Botnang,0,0,0,0,1,0,0,4,0,...,0,0,0,0,0,1,1,1,0,0


In [48]:
# For the following we keep only the total in each neighborhood

df_Stuttgart_grouped_resto['TotalFood']= df_Stuttgart_grouped_resto.sum(numeric_only=True, axis=1)
df_Stuttgart_grouped_resto=df_Stuttgart_grouped_resto[['Neighborhood', 'TotalFood']]
df_Stuttgart_grouped_resto.head()

,Neighborhood,TotalFood
0,Asemwald,9
1,Bad Cannstatt,30
2,Bergheim,12
3,Birkach,27
4,Botnang,26


Let's repeat these steps for transportation, parcs, hospital, medical center

In [49]:
# Transport
df_Stuttgart_transport = getNearbyVenues(names=nb, latitudes=lt, longitudes=lg, radius=rd, categoryId='4d4b7105d754a06379d81259')

Done


In [50]:
# Outdoor
df_Stuttgart_outdoor = getNearbyVenues(names=nb, latitudes=lt, longitudes=lg, radius=rd, categoryId='4d4b7105d754a06377d81259')

Done


In [51]:
# Working places
df_Stuttgart_work = getNearbyVenues(names=nb, latitudes=lt, longitudes=lg, radius=rd, categoryId='4d4b7105d754a06375d81259')

Done


In [52]:
# Entertainment
df_Stuttgart_fun = getNearbyVenues(names=nb, latitudes=lt, longitudes=lg, radius=rd, categoryId='4d4b7104d754a06370d81259')

Done


In [53]:
# Health
df_Stuttgart_medic = getNearbyVenues(names=nb, latitudes=lt, longitudes=lg, radius=rd, categoryId='4bf58dd8d48988d104941735')

Done


In [29]:
categoryId='4d4b7104d754a06370d81259'

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lt, 
    lg, 
    rd,
    categoryId)
results = requests.get(url).json()
results

{'meta': {'code': 400,
  'errorType': 'param_error',
  'errorDetail': 'll must be of the form XX.XX,YY.YY (received 0     48.7260\n1     48.8056\n2     48.8013\n3     48.7082\n4     48.7762\n5     48.8137\n6     48.7506\n7     48.7237\n8     48.7195\n9     48.7124\n10    48.8113\n11    48.6836\n12    48.7607\n13    48.8363\n14    48.8067\n15    48.8151\n16    48.7586\n17    48.7463\n18    48.7425\n19    48.7118\n20    48.7383\n21    48.7532\n22    48.7931\n23    48.7258\n24    48.8424\n25    48.8213\n26    48.8357\n27    48.7646\n28    48.7010\n29    48.7360\n30    48.7172\n31    48.7618\n32    48.8341\n33    48.7833\n34    48.7325\n35    48.7474\n36    48.8106\n37    48.7446\n38    48.8493\n39    48.7153\n40    48.7759\n41    48.7974\n42    48.7726\n43    48.7644\n44    48.7582\n45    48.7762\n46    48.7802\n47    48.7249\n48    48.7687\n49    48.8143\n50    48.8125\n51    48.8409\n52    48.8413\nName: Latitude, dtype: float64,0     9.1932\n1     9.2141\n2     9.0897\n3     9.2108\n4 

In [54]:
# Let's encode these data and group by neighborhood

df_Stuttgart_onehot_transport = pd.get_dummies(df_Stuttgart_transport[['Venue Category']], prefix="", prefix_sep="")
df_Stuttgart_onehot_transport['Neighborhood'] = df_Stuttgart_transport['Neighborhood'] 
fixed_columns = [df_Stuttgart_onehot_transport.columns[-1]] + list(df_Stuttgart_onehot_transport.columns[:-1])
df_Stuttgart_onehot_transport = df_Stuttgart_onehot_transport[fixed_columns]
df_Stuttgart_grouped_transport = df_Stuttgart_onehot_transport.groupby('Neighborhood').sum().reset_index()
df_Stuttgart_grouped_transport.head()

# For the following we keep only the total in each neighborhood

df_Stuttgart_grouped_transport['TotalTransport']= df_Stuttgart_grouped_transport.sum(numeric_only=True, axis=1)
df_Stuttgart_grouped_transport=df_Stuttgart_grouped_transport[['Neighborhood', 'TotalTransport']]
df_Stuttgart_grouped_transport.head()

,Neighborhood,TotalTransport
0,Asemwald,3
1,Bad Cannstatt,25
2,Bergheim,7
3,Birkach,27
4,Botnang,14


In [55]:
# Let's encode these data and group by neighborhood

df_Stuttgart_onehot_work = pd.get_dummies(df_Stuttgart_work[['Venue Category']], prefix="", prefix_sep="")
df_Stuttgart_onehot_work['Neighborhood'] = df_Stuttgart_work['Neighborhood'] 
fixed_columns = [df_Stuttgart_onehot_work.columns[-1]] + list(df_Stuttgart_onehot_work.columns[:-1])
df_Stuttgart_onehot_work = df_Stuttgart_onehot_work[fixed_columns]
df_Stuttgart_grouped_work = df_Stuttgart_onehot_work.groupby('Neighborhood').sum().reset_index()
df_Stuttgart_grouped_work.head()

# For the following we keep only the total in each neighborhood

df_Stuttgart_grouped_work['TotalWork']= df_Stuttgart_grouped_work.sum(numeric_only=True, axis=1)
df_Stuttgart_grouped_work=df_Stuttgart_grouped_work[['Neighborhood', 'TotalWork']]
df_Stuttgart_grouped_work.head()

,Neighborhood,TotalWork
0,Asemwald,8
1,Bad Cannstatt,29
2,Bergheim,14
3,Birkach,25
4,Botnang,28


In [56]:
# Let's encode these data and group by neighborhood

df_Stuttgart_onehot_outdoor = pd.get_dummies(df_Stuttgart_outdoor[['Venue Category']], prefix="", prefix_sep="")
df_Stuttgart_onehot_outdoor['Neighborhood'] = df_Stuttgart_outdoor['Neighborhood'] 
fixed_columns = [df_Stuttgart_onehot_outdoor.columns[-1]] + list(df_Stuttgart_onehot_outdoor.columns[:-1])
df_Stuttgart_onehot_outdoor = df_Stuttgart_onehot_outdoor[fixed_columns]
df_Stuttgart_grouped_outdoor = df_Stuttgart_onehot_outdoor.groupby('Neighborhood').sum().reset_index()
df_Stuttgart_grouped_outdoor.head()

# For the following we keep only the total in each neighborhood

df_Stuttgart_grouped_outdoor['TotalOutdoor']= df_Stuttgart_grouped_outdoor.sum(numeric_only=True, axis=1)
df_Stuttgart_grouped_outdoor=df_Stuttgart_grouped_outdoor[['Neighborhood', 'TotalOutdoor']]
df_Stuttgart_grouped_outdoor.head()

,Neighborhood,TotalOutdoor
0,Asemwald,6
1,Bad Cannstatt,27
2,Bergheim,16
3,Birkach,18
4,Botnang,23


In [57]:
# Let's encode these data and group by neighborhood

df_Stuttgart_onehot_medic = pd.get_dummies(df_Stuttgart_medic[['Venue Category']], prefix="", prefix_sep="")
df_Stuttgart_onehot_medic['Neighborhood'] = df_Stuttgart_medic['Neighborhood'] 
fixed_columns = [df_Stuttgart_onehot_medic.columns[-1]] + list(df_Stuttgart_onehot_medic.columns[:-1])
df_Stuttgart_onehot_medic = df_Stuttgart_onehot_medic[fixed_columns]
df_Stuttgart_grouped_medic = df_Stuttgart_onehot_medic.groupby('Neighborhood').sum().reset_index()
df_Stuttgart_grouped_medic.head()

# For the following we keep only the total in each neighborhood

df_Stuttgart_grouped_medic['TotalMedic']= df_Stuttgart_grouped_medic.sum(numeric_only=True, axis=1)
df_Stuttgart_grouped_medic=df_Stuttgart_grouped_medic[['Neighborhood', 'TotalMedic']]
df_Stuttgart_grouped_medic.head()

,Neighborhood,TotalMedic
0,Asemwald,1
1,Bad Cannstatt,20
2,Bergheim,5
3,Birkach,8
4,Botnang,12


In [58]:
df_Stuttgart_grouped=pd.concat([df_Stuttgart_grouped_resto,df_Stuttgart_grouped_outdoor['TotalOutdoor'],
                                df_Stuttgart_grouped_work['TotalWork'],
                               df_Stuttgart_grouped_transport['TotalTransport'],
                               df_Stuttgart_grouped_medic['TotalMedic']
                               ], axis=1, sort=False)

In [59]:
df_Stuttgart_grouped

,Neighborhood,TotalFood,TotalOutdoor,TotalWork,TotalTransport,TotalMedic
0,Asemwald,9,6,8,3,1.0
1,Bad Cannstatt,30,27,29,25,20.0
2,Bergheim,12,16,14,7,5.0
3,Birkach,27,18,25,27,8.0
4,Botnang,26,23,28,14,12.0
5,Burgholzhof,30,27,28,22,21.0
6,Büsnau,6,9,2,4,1.0
7,Degerloch,29,27,27,23,15.0
8,Dürrlewang,30,22,27,23,26.0
9,Fasanenhof,26,16,23,7,2.0


In [60]:
df_Stuttgart_grouped=df_Stuttgart_grouped.fillna(0)

,TotalOutdoor,TotalWork,TotalTransport
0,2.0,11.0,1.0
1,1.0,1.0,1.0
2,1.0,6.0,1.0
3,1.0,1.0,1.0
4,1.0,1.0,3.0
5,1.0,2.0,1.0
6,1.0,1.0,2.0
7,2.0,1.0,1.0
8,2.0,1.0,2.0
9,2.0,1.0,2.0


In [61]:
df_Stuttgart_grouped[df_Stuttgart_grouped.columns[1:]]=df_Stuttgart_grouped[df_Stuttgart_grouped.columns[1:]].div(df_Stuttgart_grouped[df_Stuttgart_grouped.columns[1:]].sum(axis=1), axis=0)

In [62]:
num_top_venues = 4

for hood in df_Stuttgart_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_Stuttgart_grouped[df_Stuttgart_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Asemwald----
            venue  freq
0       TotalFood  0.33
1       TotalWork  0.30
2    TotalOutdoor  0.22
3  TotalTransport  0.11


----Bad Cannstatt----
            venue  freq
0       TotalFood  0.23
1       TotalWork  0.22
2    TotalOutdoor  0.21
3  TotalTransport  0.19


----Bergheim----
            venue  freq
0    TotalOutdoor  0.30
1       TotalWork  0.26
2       TotalFood  0.22
3  TotalTransport  0.13


----Birkach----
            venue  freq
0       TotalFood  0.26
1  TotalTransport  0.26
2       TotalWork  0.24
3    TotalOutdoor  0.17


----Botnang----
            venue  freq
0       TotalWork  0.27
1       TotalFood  0.25
2    TotalOutdoor  0.22
3  TotalTransport  0.14


----Burgholzhof----
            venue  freq
0       TotalFood  0.23
1       TotalWork  0.22
2    TotalOutdoor  0.21
3  TotalTransport  0.17


----Büsnau----
            venue  freq
0    TotalOutdoor  0.41
1       TotalFood  0.27
2  TotalTransport  0.18
3       TotalWork  0.09


----Degerloch----
     

In [63]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [120]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_Stuttgart_grouped['Neighborhood']

for ind in np.arange(df_Stuttgart_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_Stuttgart_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Asemwald,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
1,Bad Cannstatt,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
2,Bergheim,TotalOutdoor,TotalWork,TotalFood,TotalTransport,TotalMedic
3,Birkach,TotalTransport,TotalFood,TotalWork,TotalOutdoor,TotalMedic
4,Botnang,TotalWork,TotalFood,TotalOutdoor,TotalTransport,TotalMedic


In [121]:
# set number of clusters
kclusters = 4

df_Stuttgart_grouped_clustering = df_Stuttgart_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_Stuttgart_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([3, 1, 0, 1, 0, 1, 0, 1, 1, 3, 1, 3, 1, 0, 0, 3, 0, 2, 0, 1],
      dtype=int32)

In [122]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [123]:
df_Stuttgart_merged = df_Stuttgart

# merge Stuttgart_grouped with Stuttgart_data to add latitude/longitude for each neighborhood
df_Stuttgart_merged = df_Stuttgart_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

df_Stuttgart_merged=df_Stuttgart_merged.fillna(0)
df_Stuttgart_merged['Cluster Labels']=df_Stuttgart_merged['Cluster Labels'].astype(int)

df_Stuttgart_merged # check the last columns!


,PostalCode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,70599,Asemwald,48.7260,9.1932,3,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
1,"70191, 70372, 70374, 70376",Bad Cannstatt,48.8056,9.2141,1,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
2,70499,Bergheim,48.8013,9.0897,0,TotalOutdoor,TotalWork,TotalFood,TotalTransport,TotalMedic
3,70599,Birkach,48.7082,9.2108,1,TotalTransport,TotalFood,TotalWork,TotalOutdoor,TotalMedic
4,70195,Botnang,48.7762,9.1304,0,TotalWork,TotalFood,TotalOutdoor,TotalTransport,TotalMedic
5,70376,Burgholzhof,48.8137,9.2125,1,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
6,70569,Büsnau,48.7506,9.0798,0,TotalOutdoor,TotalFood,TotalTransport,TotalWork,TotalMedic
7,"70567, 70597",Degerloch,48.7237,9.1534,1,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
8,70565,Dürrlewang,48.7195,9.1181,1,TotalFood,TotalWork,TotalMedic,TotalTransport,TotalOutdoor
9,70565,Fasanenhof,48.7124,9.1509,3,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic


In [124]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_Stuttgart_merged['Latitude'], df_Stuttgart_merged['Longitude'], df_Stuttgart_merged['Neighborhood'], df_Stuttgart_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [125]:
df_Stuttgart_merged.loc[df_Stuttgart_merged['Cluster Labels'] == 0, df_Stuttgart_merged.columns[[1] + list(range(5, df_Stuttgart_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Bergheim,TotalOutdoor,TotalWork,TotalFood,TotalTransport,TotalMedic
4,Botnang,TotalWork,TotalFood,TotalOutdoor,TotalTransport,TotalMedic
6,Büsnau,TotalOutdoor,TotalFood,TotalTransport,TotalWork,TotalMedic
13,Freiberg,TotalOutdoor,TotalWork,TotalFood,TotalTransport,TotalMedic
14,Giebel,TotalWork,TotalOutdoor,TotalFood,TotalTransport,TotalMedic
16,Hedelfingen,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
18,Hoffeld,TotalFood,TotalOutdoor,TotalWork,TotalTransport,TotalMedic
20,Kaltental,TotalWork,TotalOutdoor,TotalFood,TotalTransport,TotalMedic
21,Lederberg,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
23,Möhringen,TotalFood,TotalOutdoor,TotalWork,TotalTransport,TotalMedic


In [126]:
df_Stuttgart_merged.loc[df_Stuttgart_merged['Cluster Labels'] == 1, df_Stuttgart_merged.columns[[1] + list(range(5, df_Stuttgart_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Bad Cannstatt,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
3,Birkach,TotalTransport,TotalFood,TotalWork,TotalOutdoor,TotalMedic
5,Burgholzhof,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
7,Degerloch,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
8,Dürrlewang,TotalFood,TotalWork,TotalMedic,TotalTransport,TotalOutdoor
10,Feuerbach,TotalFood,TotalWork,TotalOutdoor,TotalMedic,TotalTransport
12,Frauenkopf,TotalOutdoor,TotalFood,TotalMedic,TotalTransport,TotalWork
19,Hohenheim,TotalTransport,TotalFood,TotalWork,TotalOutdoor,TotalMedic
22,Luginsland,TotalOutdoor,TotalWork,TotalMedic,TotalTransport,TotalFood
28,Plieningen,TotalWork,TotalFood,TotalTransport,TotalOutdoor,TotalMedic


In [127]:
df_Stuttgart_merged.loc[df_Stuttgart_merged['Cluster Labels'] == 2, df_Stuttgart_merged.columns[[1] + list(range(5, df_Stuttgart_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
17,Heumaden,TotalMedic,TotalWork,TotalOutdoor,TotalFood,TotalTransport
33,Rotenberg,TotalMedic,TotalOutdoor,TotalWork,TotalFood,TotalTransport
34,Schönberg,TotalMedic,TotalOutdoor,TotalWork,TotalFood,TotalTransport
38,Stammheim,TotalMedic,TotalWork,TotalFood,TotalTransport,TotalOutdoor


In [128]:
df_Stuttgart_merged.loc[df_Stuttgart_merged['Cluster Labels'] == 3, df_Stuttgart_merged.columns[[1] + list(range(5, df_Stuttgart_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Asemwald,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
9,Fasanenhof,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
11,Flughafen,TotalFood,TotalWork,TotalMedic,TotalTransport,TotalOutdoor
15,Hausen,TotalWork,TotalFood,TotalOutdoor,TotalMedic,TotalTransport
26,Neuwirtshaus,TotalWork,TotalFood,TotalTransport,TotalOutdoor,TotalMedic
30,Rohr,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
32,Rot,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
48,Wangen,TotalFood,TotalWork,TotalTransport,TotalOutdoor,TotalMedic
49,Weilimdorf,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic
50,Wolfbusch,TotalFood,TotalWork,TotalOutdoor,TotalTransport,TotalMedic


In [129]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, radii, cluster in zip(df_Stuttgart_merged['Latitude'], df_Stuttgart_merged['Longitude'], df_Stuttgart_merged['Neighborhood'], df_Stuttgart_grouped['TotalMedic'], df_Stuttgart_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=radii*60,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters